# Automação Web para Busca de Cotações de Moedas

## Entendendo a Situação Atual

Você trabalha em uma importadora e precisa atualizar diariamente uma planiha do Excel com os custos de cada produto. Esses produtos podem ter seus valores vinculados as seguintes moedas:

- Dólar
- Euro
- Ouro

Para obter a cotação dessas moedas, fazemos buscas na internet. Com os valores das cotações podemos atualizar nossos preços de venda, considerando uma margem de contribuição.

## Base de Dados

O link para a base de dados está disponível no link logo abaixo:

- https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

## Divisão do Problema

1. Obter a cotação do dólar
2. Obter a cotação do euro
3. Obter a cotação do ouro
4. Atualizar a planilha com as cotações de cada moeda
5. Atualizar o preço de compra e o preço de venda de cada produto
6. Exportar a planilha atualizada

## Criando e Configurando o Navegador

In [1]:
# Importa o webdriver a partir da biblioteca selenium
from selenium import webdriver

# Importa o gerenciador do webdriver do Google Chrome
from webdriver_manager.chrome import ChromeDriverManager

# Importa a classe Service
from selenium.webdriver.chrome.service import Service

# Faz o download do webdriver do Google Chrome
servico = Service(ChromeDriverManager().install())

# Cria um navegador do Google Chrome
navegador = webdriver.Chrome(service=servico)

In [2]:
# Importa o método de localização de elementos
from selenium.webdriver.common.by import By

# Importa o Keys para escrevermos no navegador
from selenium.webdriver.common.keys import Keys

## Solução

A função abaixa será chamada repetidas vezes para obtermos cotações de moedas.

In [3]:
def obter_cotacao_moeda(busca: str) -> float:
    '''Obtém a cotação da moeda especificada'''
    
    # Acessa o site do Google
    navegador.get('https://google.com')

    # Localiza o campo de pesquisa do Google via XPATH
    campo_pesquisa = navegador.find_element(By.XPATH, r'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input')

    # Escreve no campo de pesquisa
    campo_pesquisa.send_keys(busca)

    # Dá enter no campo de busca
    campo_pesquisa.send_keys(Keys.ENTER)

    # Obtém a cotação do dólar
    cotacao = navegador.find_element(By.XPATH, r'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
    
    # Retorna a cotação
    return float(cotacao)

A função abaixo será usada para obter a cotação do ouro.

In [4]:
def obter_cotacao_ouro():
    '''Obtém a cotação do ouro'''
    
    # Acessa o site do Melhor Câmbio
    navegador.get('https://www.melhorcambio.com/ouro-hoje')
    
    # Localiza o campo com a cotação do ouro
    campo_cotacao_ouro = navegador.find_element(By.XPATH, r'//*[@id="comercial"]')
    
    # Obtém o valor da cotação do ouro
    cotacao_ouro = campo_cotacao_ouro.get_attribute('value')
        
    # Retorna o valor da cotação do ouro
    return cotacao_ouro

### Obter a cotação do dólar

In [5]:
# Obtém a cotação do dólar
cotacao_dolar = obter_cotacao_moeda('cotação do dólar')

# Exibe o valor da cotação do dólar
print(f'Cotação do dólar: R${cotacao_dolar}')

Cotação do dólar: R$5.2332


### Obter a cotação do euro

In [6]:
# Obtém a cotação do euro
cotacao_euro = obter_cotacao_moeda('cotação do euro')

# Exibe o valor da cotação do euro
print(f'Cotação do euro: R${cotacao_euro}')

Cotação do euro: R$5.5788528600000005


### Obter a cotação do ouro

In [7]:
# Obtém a cotação do ouro
cotacao_ouro = obter_cotacao_ouro()

# Substitui a vírgula pelo ponto
cotacao_ouro = cotacao_ouro.replace(',', '.')

# Exibe o valor da cotação do ouro
print(f'Cotação do ouro: R${cotacao_ouro}')

Cotação do ouro: R$309.97


Dado que neste ponto já realizamos todas as buscas, podemos fechar o navegador:

In [8]:
navegador.quit()

### Atualizar a planilha com as cotações de cada moeda

In [9]:
# Importa o pandas com o apelido pd
import pandas as pd

# Importa a base de dados
df_produtos = pd.read_excel('Produtos.xlsx')

# Exibe a base de dados
df_produtos

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


In [10]:
# Atualiza os valores na coluna Cotação
df_produtos.loc[df_produtos['Moeda'] == 'Dólar', 'Cotação'] = float(cotacao_dolar)
df_produtos.loc[df_produtos['Moeda'] == 'Euro', 'Cotação'] = float(cotacao_euro)
df_produtos.loc[df_produtos['Moeda'] == 'Ouro', 'Cotação'] = float(cotacao_ouro)

# Exibe o dataframe
df_produtos

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.233200,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,5.578853,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5.233200,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5.233200,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,5.578853,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5.233200,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,309.970000,7000.00,1.15,8050.000


### Atualizar o preço de compra e o preço de venda de cada produto

In [11]:
# Recalcula o preço de compra
df_produtos['Preço de Compra'] = df_produtos['Preço Original'] * df_produtos['Cotação']

# Recalcula o preço de venda
df_produtos['Preço de Venda'] = df_produtos['Preço de Compra'] * df_produtos['Margem']

# Exibe o dataframe
df_produtos

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.233200,5233.147668,1.40,7326.406735
1,Carro Renault,4500.00,Euro,5.578853,25104.837870,2.00,50209.675740
2,Notebook Dell,899.99,Dólar,5.233200,4709.827668,1.70,8006.707036
3,IPhone,799.00,Dólar,5.233200,4181.326800,1.70,7108.255560
4,Carro Fiat,3000.00,Euro,5.578853,16736.558580,1.90,31799.461302
5,Celular Xiaomi,480.48,Dólar,5.233200,2514.447936,2.00,5028.895872
6,Joia 20g,20.00,Ouro,309.970000,6199.400000,1.15,7129.310000


### Exportar a planilha atualizada

In [12]:
# Exporta a planilha para o caminho especificado
df_produtos.to_excel('BaseProdutosAtualizada.xlsx')